In [1]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action='ignore',category=np.VisibleDeprecationWarning)

# Lecture 11 #

### Rows from lists

In [2]:
Table().with_columns('Numbers', np.arange(1, 4))

Numbers
1
2
3


In [3]:
drinks = Table(['Drink', 'Cafe', 'Price'])
drinks

Drink,Cafe,Price


In [4]:
drinks = drinks.with_rows([
    ['Matcha Latte', 'Northtown', 5.5],
    ['Espresso', 'Cafe Mokka',  2.75],
    ['Latte',    'Cafe Mokka',  5.25],
    ['Espresso', "Jitter Bean",   2]
])
drinks

Drink,Cafe,Price
Matcha Latte,Northtown,5.5
Espresso,Cafe Mokka,2.75
Latte,Cafe Mokka,5.25
Espresso,Jitter Bean,2


## Review: Group by one column

In [5]:
survey = Table.read_table('data111_survey_fa24.csv')
survey.show(3)

Timestamp,Pets,Handedness,Extroversion,Texts,Hours of Sleep,Sleep Position,Pant Leg,Piercings,Food Preference
2024/08/26 10:11:55 AM MDT,Dog;Cat;Bunny;Bearded Dragon,Right,4,3,9,On your right side,Right,0,Savory
2024/08/26 10:18:32 AM MDT,Dog,Right,4,8,8,On your left side,Right,2,Sweet
2024/08/26 10:52:17 AM MDT,Cat,Right,4,7,7,On your right side,Right,2,Savory


In [6]:
survey.group("Handedness")

Handedness,count
Left,9
Right,65


In [7]:
(survey
 .select('Handedness', 'Hours of Sleep')
 .group("Handedness", np.average))

Handedness,Hours of Sleep average
Left,7.16667
Right,7.12615


In [8]:
survey.group("Handedness", np.average)

Handedness,Timestamp average,Pets average,Extroversion average,Texts average,Hours of Sleep average,Sleep Position average,Pant Leg average,Piercings average,Food Preference average
Left,,,4.66667,4.66667,7.16667,,,1.77778,
Right,,,5.32308,4.33846,7.12615,,,2.04615,


## Cross-classification: grouping by two columns

In [9]:
survey.group(["Handedness", 'Sleep Position'])

Handedness,Sleep Position,count
Left,On your back,2
Left,On your left side,4
Left,On your right side,2
Left,On your stomach,1
Right,On your back,9
Right,On your left side,25
Right,On your right side,22
Right,On your stomach,9


In [10]:
(survey
 .select("Handedness", 'Sleep Position', "Hours of Sleep")
 .group(["Handedness", 'Sleep Position'], collect=np.average))

Handedness,Sleep Position,Hours of Sleep average
Left,On your back,7
Left,On your left side,7.375
Left,On your right side,6.5
Left,On your stomach,8
Right,On your back,7.05556
Right,On your left side,7.148
Right,On your right side,7.29545
Right,On your stomach,6.72222


## Pivot Tables

In [12]:
survey.pivot('Sleep Position','Handedness')

Handedness,On your back,On your left side,On your right side,On your stomach
Left,2,4,2,1
Right,9,25,22,9


In [13]:
survey.pivot('Sleep Position','Handedness', values = 'Hours of Sleep', collect = np.average)

Handedness,On your back,On your left side,On your right side,On your stomach
Left,7,7.375,6.5,8
Right,7.05556,7.148,7.29545,6.72222


In [14]:
# This cell will throw an error
# You cannot include just 1 of the optional arguments
survey.pivot('Sleep Position', 
             'Handedness', 
             collect=np.average)

TypeError: collect requires values to be specified

## Joins ##

In [15]:
drinks

Drink,Cafe,Price
Matcha Latte,Northtown,5.5
Espresso,Cafe Mokka,2.75
Latte,Cafe Mokka,5.25
Espresso,Jitter Bean,2


In [16]:
discounts = Table().with_columns(
    'Coupon % off', make_array(10, 25, 5),
    'Location', make_array('Northtown', 'Cafe Mokka', 'Northtown')
)
discounts

Coupon % off,Location
10,Northtown
25,Cafe Mokka
5,Northtown


In [17]:
combined = drinks.join('Cafe',discounts,'Location')
combined

Cafe,Drink,Price,Coupon % off
Cafe Mokka,Espresso,2.75,25
Cafe Mokka,Latte,5.25,25
Northtown,Matcha Latte,5.5,10
Northtown,Matcha Latte,5.5,5


In [18]:
discounted_frac = 1 - combined.column('Coupon % off') / 100
combined.with_column(
    'Discounted Price', 
    combined.column('Price') * discounted_frac
)

Cafe,Drink,Price,Coupon % off,Discounted Price
Cafe Mokka,Espresso,2.75,25,2.0625
Cafe Mokka,Latte,5.25,25,3.9375
Northtown,Matcha Latte,5.5,10,4.95
Northtown,Matcha Latte,5.5,5,5.225


In [19]:
drinks.join('Cafe',drinks,'Cafe')

Cafe,Drink,Price,Drink_2,Price_2
Cafe Mokka,Espresso,2.75,Espresso,2.75
Cafe Mokka,Espresso,2.75,Latte,5.25
Cafe Mokka,Latte,5.25,Espresso,2.75
Cafe Mokka,Latte,5.25,Latte,5.25
Jitter Bean,Espresso,2,Espresso,2
Northtown,Matcha Latte,5.5,Matcha Latte,5.5


## Activity ##

In [20]:
# From the CORGIS Dataset Project
# By Austin Cory Bart acbart@vt.edu
# Version 2.0.0, created 3/22/2016
# https://corgis-edu.github.io/corgis/csv/skyscrapers/

sky = Table.read_table('skyscrapers.csv')
sky = (sky.with_column('age', 2024 - sky.column('completed'))
          .drop('completed'))
sky.show(3)

name,material,city,height,age
One World Trade Center,mixed/composite,New York City,541.3,10
Willis Tower,steel,Chicago,442.14,50
432 Park Avenue,concrete,New York City,425.5,9


1. For each city, how tall is the tallest building for each material?

In [23]:
# 1. For each city, how tall is the tallest building for each material?

sky.pivot('material','city','height',max)

city,concrete,mixed/composite,steel
Atlanta,264.25,311.8,169.47
Austin,208.15,0,93.6
Baltimore,161.24,0,155.15
Boston,121.92,139,240.79
Charlotte,265.48,239.7,179.23
Chicago,423.22,306.94,442.14
Cincinnati,125,202.69,175
Cleveland,125,288.65,215.8
Columbus,79.25,0,169.3
Dallas,176.48,280.72,270.06


In [24]:
sky.select('city','material','height').group(['city','material'],max)

city,material,height max
Atlanta,concrete,264.25
Atlanta,mixed/composite,311.8
Atlanta,steel,169.47
Austin,concrete,208.15
Austin,steel,93.6
Baltimore,concrete,161.24
Baltimore,steel,155.15
Boston,concrete,121.92
Boston,mixed/composite,139
Boston,steel,240.79


2. For each city, what’s the height difference between the tallest steel building and the tallest concrete building?

In [ ]:
# 2. For each city, what’s the height difference between the tallest 
#    steel building and the tallest concrete building?









3. Generate a table of the names of the oldest buildings for each material for each city:

In [ ]:
# 3. Generate a table of the names of the oldest buildings for each 
#    material for each city:

# Hint: You can use sort to find the name of the oldest building in the dataset
sky.sort('age', descending=True).column('name').item(0)


# Put your solution here






